In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
sample = np.array([[0.88645553, 5.84095018],
        [1.76306821, 1.28529014],
        [2.09187193, 6.06022346],
        [2.7455983 , 4.10650061],
        [1.156449  , 0.05270523],
        [2.41036002, 4.1243531 ],
        [1.85975128, 6.06105961],
        [1.21447324, 4.50789405],
        [2.98670697, 2.76236853],
        [0.85240889, 4.25292296],
        [2.51060472, 0.16869898],
        [1.62333082, 0.1550452 ],
        [2.48189743, 3.74707815],
        [1.57124183, 4.57583136],
        [0.97387373, 5.09057019],
        [1.48945306, 1.62557981],
        [1.63373765, 4.45798143],
        [2.59893856, 1.05395551],
        [1.55004825, 3.8295616 ],
        [2.53605197, 2.37092827]])

In [9]:
def average_fidelity(gate_list, wire_list, noise_param, over_rot):

    """This function returns the average fidelity of a noisy superconducting circuit
    with respect to the ideal version of such circuit, which contains CNOT gates.

    Args:
        - gate_list (list(str)): Gates, assumed to be non-parametric, contained in the ideal circuit. 
        - wire_list (list(int)): Wires on which each of the gates in gate_list act
        - noise_param (float): The noise parameter characterizing the depolarizing gate after the
          sqrt(iSWAP) gates
        - over_rot (float): Extra rotation angle on each rotation gate.
    Returns: 
        - (float): Average fidelity of the superconducting circuit with respect to the ideal circuit.
    """

    
    def circuit():
        """This function creates a circuit from gate_list and wire_list (You shouldn't modify this)"""
        for i in range(len(gate_list)):
            getattr(qml,gate_list[i])(wires=wire_list[i])
        
    ref_device = qml.device('default.mixed', wires = 2)

    @qml.qnode(ref_device)
    def reference_circuit(phi, theta):
        """Ideal circuit, with prior state preparation (You shouldn't modify this)"""
        qml.Rot(phi, theta, 0, wires = 0)
        circuit()
        return qml.state()
    
    dev = qml.device('default.mixed', wires = 2)
    
    def custom_cnot(wires):
        # Put your code here #
        # Return your custom CNOT gate as implemented by a superconducting qubit as an array, including noise
        # See documentation for more details!
        angle = math.pi/2 + over_rot
        return [
            qml.RZ(-math.pi/2 + over_rot, wires=wires[0]),
            qml.RX(angle, wires=wires[1]),
            qml.RZ(angle, wires=wires[1]),
            qml.ISWAP(wires=wires),
            qml.RX(angle, wires=wires[0]),
            qml.ISWAP(wires=wires),
            qml.RZ(angle, wires=wires[1]),
            qml.DepolarizingChannel(noise_param, wires=wires[0]),
            qml.DepolarizingChannel(noise_param, wires=wires[1])
        ]
    custom_decomps = {qml.CNOT : custom_cnot}# Write here the custom decomposition of the CNOT gate for your device.
    expand_fn = qml.transforms.create_decomp_expand_fn(custom_decomps, dev)
    dev.custom_expand(expand_fn) 
    
    @qml.qnode(dev, expansion_strategy = "device")
    def superconducting_circuit(phi,theta):
        qml.Rot(phi, theta, 0, wires = 0)
        circuit()
        return qml.state()
    
    def fidelity(phi,theta):
        """Return the fidelity of the final state of the circuit, with respect to the ideal circuit, given
        an initial state parametrized by phi and theta"""
        # Put your code here #
        fid = qml.math.fidelity(reference_circuit(phi, theta), superconducting_circuit(phi, theta))
        
        return fid

    # Write any other helper functions here, if you need any.
    fidelisty_list = []
    for theta, phi in sample:
        fidelisty_list.append(fidelity(phi, theta))
    
    # Return the average fidelity by running over the sample states given to you in the list "sample"
    print(fidelisty_list, np.mean(fidelisty_list))
    return np.mean(fidelisty_list)

In [10]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
		
    ins = json.loads(test_case_input)
    output = average_fidelity(*ins)

    return str(output) 

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises: 
            ``AssertionError`` if the solution output is incorrect in any way.
    """

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-2
    ), "Your calculated average fidelity isn't quite right."


test_cases = [['[["CNOT","PauliZ","PauliZ"],[[1,0],1,0],0.0001,0.01]', '0.445426'], ['[["Hadamard","CNOT","PauliZ"],[0,[0,1],0],0.0001,0.01]', '0.363157']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[["CNOT","PauliZ","PauliZ"],[[1,0],1,0],0.0001,0.01]'...
[0.3554257322875711, 0.37115215780081395, 0.4374951882238408, 0.6488944645739162, 0.34452872674985535, 0.5338380745590995, 0.3865504664008607, 0.3431108260249494, 0.7234566795108207, 0.35696051027070724, 0.5680250306678444, 0.3550067720862925, 0.5581409439792562, 0.3507625352732439, 0.3514787670483292, 0.3459090385727139, 0.35596704842791127, 0.5986473533081637, 0.34929763173044515, 0.5768264632033162] 0.44557372053499755
Correct!
Running test case 1 with input '[["Hadamard","CNOT","PauliZ"],[0,[0,1],0],0.0001,0.01]'...
[0.36839487275051824, 0.3486588401882622, 0.335734736291983, 0.38910599439629334, 0.3694574749227377, 0.3431040882076092, 0.3438518436057919, 0.36894549816498945, 0.45048190231907986, 0.36804946762752794, 0.3521877201505376, 0.35574585783851354, 0.34920055282983387, 0.35821944863855776, 0.36918266828216567, 0.3617414436354856, 0.3552352357227949, 0.3634141720333929, 0.35917929407161

In [24]:
math.pi/2

1.5707963267948966